In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import datetime 
from dateutil.parser import parse
import nbformat
from functools import reduce

In [ ]:
#defining default folder structure
RAW_DATA_FOLDER = 'data/raw/'
OUTPUT_DATA_FOLDER = 'data/output/'
REPORT_FOLDER = 'reports/'

#sectors and countries that we are interested
sector_list = ['Energy Storage & Management', 'Industry 4.0', 'Mobility & Automotive', 'Supply Chain & Logistics', 'Other']
country_list = ['Germany', 'Turkey', 'United Kingdom', 'Other']

#import the dataset
pipeline = pd.read_csv(os.path.join(RAW_DATA_FOLDER, 'pipeline.csv'))
#set_indexes([energy, industry, sc_logistics, mobility], ["id", "id", "id", "id"])

#prints a cumulative histogram if TRUE
CUMULATIVE = False


In [ ]:
#standardize the date input
pipeline.rename(columns={'Day Created': 'Date'}, inplace=True)
pipeline.loc[~pipeline['Country'].isin(country_list), 'Country'] = 'Other'
#get rid of irrelevant rows
pipeline.dropna(subset = ['Sector'], inplace=True)
pipeline.replace("-", np.nan, inplace=True)

In [ ]:
pipeline[pipeline["Term Sheet Date"].notnull()]["Term Sheet Date"]

In [ ]:
#adjust time format
pipeline["Date"] = pd.to_datetime(pipeline["Date"]).dt.strftime('%Y-%m-%d')
pipeline["Date"] = pipeline["Date"].apply(lambda x : parse(x))
pipeline["One Pager Date"] = pd.to_datetime(pipeline["One Pager Date"]).dt.strftime('%Y-%m-%d')
pipeline["One Pager Date"] = pipeline[pipeline["One Pager Date"].notnull()]['One Pager Date'].apply(lambda x : parse(x))
pipeline["Term Sheet Date"] = pd.to_datetime(pipeline["Term Sheet Date"]).dt.strftime('%Y-%m-%d')
pipeline["Term Sheet Date"] = pipeline[pipeline["Term Sheet Date"].notnull()]['Term Sheet Date'].apply(lambda x : parse(x))
pipeline["Closing Date"] = pd.to_datetime(pipeline["Closing Date"]).dt.strftime('%Y-%m-%d')
pipeline["Closing Date"] = pipeline[pipeline["Closing Date"].notnull()]['Closing Date'].apply(lambda x : parse(x))

In [ ]:
#array(['First Look', 'Term Sheet', 'One Pager', 'Invested'], dtype=object)
invested_mask = pipeline['First Look'] == 'Invested'
invested_count = pipeline[invested_mask][['Name']].count()

term_sheet_mask = invested_mask | (pipeline['First Look'] == 'Term Sheet')
term_sheet_count = pipeline[term_sheet_mask][['Name']].count()

one_pager_mask = term_sheet_mask | (pipeline['First Look'] == 'One Pager')
one_pager_count = pipeline[one_pager_mask][['Name']].count()

first_look_mask = one_pager_mask | (pipeline['First Look'] == 'First Look')
first_look_count = pipeline[first_look_mask][['Name']].count()

In [ ]:
#test the mask feature
pipeline[one_pager_mask]['Name'].unique()

In [ ]:
#Quarter Creator
pipeline["First Look Quarter"] = pipeline[first_look_mask]["Date"].dt.to_period("Q")
pipeline["One Pager Quarter"] = pipeline[one_pager_mask]["One Pager Date"].dt.to_period("Q")
pipeline["Term Sheet Quarter"] = pipeline[term_sheet_mask]["Term Sheet Date"].dt.to_period("Q")
pipeline["Invested Quarter"] = pipeline[invested_mask]["Closing Date"].dt.to_period("Q")

In [ ]:
if CUMULATIVE:
    invested_quarter = pipeline.groupby('Invested Quarter').size().cumsum().to_frame('Invested').reset_index().rename(columns={'Invested Quarter': 'Quarter'})
    term_sheet_quarter = pipeline.groupby('Term Sheet Quarter').size().cumsum().to_frame('Term Sheet').reset_index().rename(columns={'Term Sheet Quarter': 'Quarter'})
    one_pager_quarter = pipeline.groupby('One Pager Quarter').size().cumsum().to_frame('One Pager').reset_index().rename(columns={'One Pager Quarter': 'Quarter'})
    first_look_quarter = pipeline.groupby('First Look Quarter').size().cumsum().to_frame('First Look').reset_index().rename(columns={'First Look Quarter': 'Quarter'})
else:
    invested_quarter = pipeline.groupby('Invested Quarter').size().to_frame('Invested').reset_index().rename(columns={'Invested Quarter': 'Quarter'})
    term_sheet_quarter = pipeline.groupby('Term Sheet Quarter').size().to_frame('Term Sheet').reset_index().rename(columns={'Term Sheet Quarter': 'Quarter'})
    one_pager_quarter = pipeline.groupby('One Pager Quarter').size().to_frame('One Pager').reset_index().rename(columns={'One Pager Quarter': 'Quarter'})
    first_look_quarter = pipeline.groupby('First Look Quarter').size().to_frame('First Look').reset_index().rename(columns={'First Look Quarter': 'Quarter'})

In [ ]:
#check the value
print(""" 
first_look: {}
one_pager: {}
term_sheet: {} 
invested: {}
""".format(int(first_look_count), int(one_pager_count), int(term_sheet_count), int(invested_count)))

In [ ]:
dfs = [invested_quarter, term_sheet_quarter, one_pager_quarter, first_look_quarter]

In [ ]:
#merge all dfs on Quarter column
df_final = reduce(lambda left,right: pd.merge(left, right, on='Quarter', how='outer'), dfs)

In [ ]:
#replace NaN values with 0
df_final = df_final.replace(np.nan, 0)

In [ ]:
#column type transformation
df_final['Invested'] = df_final['Invested'].astype('int')
df_final['Term Sheet'] = df_final['Term Sheet'].astype('int')
df_final['One Pager'] = df_final['One Pager'].astype('int')
df_final['First Look'] = df_final['First Look'].astype('int')

In [ ]:
df_final.sort_values('Quarter', inplace=True)

In [ ]:
df_final

In [ ]:
## time to visualize the data
# Values of each group
bars1 = list(df_final['One Pager'].values)
bars2 = list(df_final['Invested'].values)
bars3 = list(df_final['Term Sheet'].values)

# Heights of bars1 + bars2
bars = np.add(bars1, bars2).tolist()
 
# The position of the bars on the x-axis
r = list(np.arange(13))
 
# Names of group and bar width
names = list(df_final['Quarter'].astype(str))
barWidth = 0.35
 
# Create brown bars
plt.bar(r, bars1, color='red', edgecolor='white', width=barWidth)
# Create green bars (middle), on top of the firs ones
plt.bar(r, bars2, bottom=bars1, color='green', edgecolor='white', width=barWidth)
# Create blue bars (top)
plt.bar(r, bars3, bottom=bars, color='blue', edgecolor='white', width=barWidth)
 
# Custom X axis
plt.xticks(r, names, fontweight='bold',rotation='vertical')

plt.xlabel("Quarter")
plt.ylabel("Number")
plt.title("Number of Stages in Each Quarter")

# Custom Legend
one_pager = mpatches.Patch(color='red', label='One Pager')
invested = mpatches.Patch(color='green', label='Invested')
term_sheet = mpatches.Patch(color='blue', label='Term Sheet')
plt.legend(handles=[one_pager,invested,term_sheet], loc=2)

plt.ylim([0, 10])

# Show graphic
plt.show()
